In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import TensorBoard
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Implement from https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

auc_roc = as_keras_metric(tf.metrics.auc)
recall = as_keras_metric(tf.metrics.recall)
precision = as_keras_metric(tf.metrics.precision)


# dimensions of our images.
img_width, img_height = 250, 250

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 60

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[auc_roc,recall,precision,'accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

tensorboard = TensorBoard(log_dir='./graph', histogram_freq=0,  
          write_graph=True, write_images=True)    

 


Found 354 images belonging to 5 classes.
Found 33 images belonging to 5 classes.


In [3]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[tensorboard],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
model.save('classify.h5')
model.save_weights('classify_weight.h5')

Epoch 1/50
33/33 [==============================] - 60s 2s/step - loss: -12.4386 - auc: 0.4695 - recall: 0.9697 - precision: 0.7934 - acc: 0.2914 - val_loss: -17.8748 - val_auc: 0.4941 - val_recall: 1.0000 - val_precision: 0.8210 - val_acc: 0.2424
Epoch 2/50
33/33 [==============================] - 56s 2s/step - loss: -13.0014 - auc: 0.4961 - recall: 1.0000 - precision: 0.8223 - acc: 0.2902 - val_loss: -17.8748 - val_auc: 0.4972 - val_recall: 1.0000 - val_precision: 0.8226 - val_acc: 0.2424
Epoch 3/50
33/33 [==============================] - 56s 2s/step - loss: -12.8715 - auc: 0.4978 - recall: 1.0000 - precision: 0.8235 - acc: 0.2911 - val_loss: -17.8748 - val_auc: 0.4982 - val_recall: 1.0000 - val_precision: 0.8235 - val_acc: 0.2424
Epoch 4/50
33/33 [==============================] - 56s 2s/step - loss: -13.0797 - auc: 0.4985 - recall: 1.0000 - precision: 0.8240 - acc: 0.2901 - val_loss: -17.8748 - val_auc: 0.4987 - val_recall: 1.0000 - val_precision: 0.8240 - val_acc: 0.2424
Epoch 5/

Epoch 34/50
33/33 [==============================] - 56s 2s/step - loss: -12.9269 - auc: 0.4998 - recall: 1.0000 - precision: 0.8252 - acc: 0.2914 - val_loss: -17.8748 - val_auc: 0.4998 - val_recall: 1.0000 - val_precision: 0.8252 - val_acc: 0.2424
Epoch 35/50
33/33 [==============================] - 56s 2s/step - loss: -12.9566 - auc: 0.4999 - recall: 1.0000 - precision: 0.8252 - acc: 0.2975 - val_loss: -17.8748 - val_auc: 0.4999 - val_recall: 1.0000 - val_precision: 0.8253 - val_acc: 0.2424
Epoch 36/50
33/33 [==============================] - 56s 2s/step - loss: -12.9651 - auc: 0.4999 - recall: 1.0000 - precision: 0.8252 - acc: 0.2841 - val_loss: -17.8748 - val_auc: 0.4999 - val_recall: 1.0000 - val_precision: 0.8252 - val_acc: 0.2424
Epoch 37/50
33/33 [==============================] - 57s 2s/step - loss: -13.1370 - auc: 0.4999 - recall: 1.0000 - precision: 0.8252 - acc: 0.2888 - val_loss: -17.8748 - val_auc: 0.4999 - val_recall: 1.0000 - val_precision: 0.8252 - val_acc: 0.2424
Epoc